In [1]:
from lsst.summit.utils.bestEffort import BestEffortIsr
bestEffort = BestEffortIsr(embargo=True)

In [2]:
dataId = {"day_obs": 20231212, "seq_num": 652, "detector": 0}

In [3]:
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

In [4]:
exp = bestEffort.getExposure(dataId)
record = butlerUtils.getExpRecordFromDataId(butler, dataId)

In [5]:
from importlib import reload
import lsst.rubintv.production.rubinTv
reload(lsst.rubintv.production.rubinTv)
#from lsst.rubintv.production import CalibrateCcdRunner
from lsst.rubintv.production.utils import checkRubinTvExternalPackages, LocationConfig
from lsst.summit.utils.utils import setupLogging

setupLogging()
checkRubinTvExternalPackages()

location = 'slac_testing'
locationConfig = LocationConfig(location)
print(f'Running CalibrateCcd runner at {location}...')
channel = lsst.rubintv.production.rubinTv.CalibrateCcdRunner(locationConfig=locationConfig,
                             instrument='LATISS',
                             embargo=True
                             )

channel.callback(record)

Running CalibrateCcd runner at slac_testing...


  if butlerUtils.datasetExists(self.butler, self.dataProduct, dataId):

ERROR:lsst.rubintv.production.auxtel_calibrateCcd:No reference tables could be found for input region
Traceback (most recent call last):
  File "/sdf/group/rubin/u/yusra/DM-41929/rubintv_production/python/lsst/rubintv/production/rubinTv.py", line 862, in callback
    calibrateRes = self.calibrate.run(charRes.exposure,
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/utils/gd9c83c13ba+81bc2a20b4/python/lsst/utils/timer.py", line 300, in timeMethod_wrapper
    res = func(self, *args, **keyArgs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/pipe_tasks/gd151e6f194+8426de5ab5/python/lsst/pipe/tasks/calibrate.py", line 603, in run
    astromRes = self.astrometry.run(
                ^^^^^^^^^^^^^^^^^^^^
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linu

I confirmed that ^^^ works if I switch to "gaia_dr2_20200414" but I want to try to get dr3 working first, especially because looking at the RubinTV history it looks like CalibrateCcdRunner DID WORK for a period after we switched to gaia_dr3 on Sept 15. 

### So why can't it find "gaia_dr3_20230707"?

I confirmed that this works fine with  d_2023_12_13 
```
pipetask run -b /repo/embargo -p $DRP_PIPE_DIR/pipelines/LATISS/DRP.yaml#isr,characterizeImage,calibrate -i LATISS/raw/all,LATISS/calib,LATISS/runs/quickLook  -o u/yusra/DM-41929 -d "exposure.day_obs = 20231212 AND exposure.seq_num=652
```

In [6]:
# nothing..
set(butler.registry.queryDatasets("gaia_dr3_20230707", dataId=channel.getVisitDataId(record)).expanded())

set()

In [7]:
# something...
set(butler.registry.queryDatasets("gaia_dr2_20200414", dataId=channel.getVisitDataId(record)).expanded())

{DatasetRef(DatasetType('gaia_dr2_20200414', {htm7}, SimpleCatalog), {htm7: 162888}, run='refcats/DM-28636', id=2ba04f87-dbc5-473c-8362-782525f8549c)}

In [8]:
# and same if I make a butler from scratch
from lsst.daf.butler import Butler
plainButler = Butler("/repo/embargo", collections=["LATISS/raw/all","LATISS/calib","LATISS/runs/quickLook"])
set(plainButler.registry.queryDatasets("gaia_dr3_20230707", dataId=channel.getVisitDataId(record)).expanded())

set()

In [9]:
set(plainButler.registry.queryDatasets("gaia_dr2_20200414", dataId=channel.getVisitDataId(record)).expanded())

{DatasetRef(DatasetType('gaia_dr2_20200414', {htm7}, SimpleCatalog), {htm7: 162888}, run='refcats/DM-28636', id=2ba04f87-dbc5-473c-8362-782525f8549c)}

## Appendix. Already resolved.
### Compare configs


In [10]:
goodRun  = Butler("/repo/embargo", collections=["u/yusra/DM-41929"])
config = goodRun.get("calibrate_config")

In [13]:
def print_diff_keys_recursive(dict1, dict2, parent_key=''):
    for key in dict1:
        # Constructing the full key path
        full_key = f"{parent_key}.{key}" if parent_key else key

        # Check if the key is in the second dictionary
        if key in dict2:
            if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
                # If both values are dictionaries, recurse
                print_diff_keys_recursive(dict1[key], dict2[key], full_key)
            elif dict1[key] != dict2[key]:
                # If values are different and not dictionaries, print the key
                print(full_key, dict1[key],  dict2[key])
        else:
            # If key is not in dict2, print the key
            print(full_key)

In [14]:
print_diff_keys_recursive(config.toDict(), channel.calibrate.config.toDict())

astrometry.sourceSelector.values.science.doIsolated True False
astrometry.sourceSelector.values.science.doRequirePrimary True False
doDeblend True False
measurement.slots.shape ext_shapeHSM_HsmSourceMoments base_SdssShape
measurement.slots.psfShape ext_shapeHSM_HsmPsfMoments base_SdssShape_psf
measurement.plugins.names ['ext_shapeHSM_HsmPsfMoments', 'base_GaussianFlux', 'ext_photometryKron_KronFlux', 'base_Blendedness', 'base_SdssShape', 'base_NaiveCentroid', 'ext_shapeHSM_HsmSourceMomentsRound', 'base_SdssCentroid', 'base_PixelFlags', 'base_LocalBackground', 'ext_shapeHSM_HsmShapeRegauss', 'base_CircularApertureFlux', 'ext_shapeHSM_HsmPsfMomentsDebiased', 'ext_shapeHSM_HsmSourceMoments', 'base_Jacobian', 'base_SkyCoord', 'base_Variance', 'base_CompensatedGaussianFlux', 'base_FPPosition', 'base_PsfFlux'] ['base_LocalBackground', 'base_Jacobian', 'base_GaussianFlux', 'base_Variance', 'base_CircularApertureFlux', 'base_SdssShape', 'base_SkyCoord', 'base_CompensatedGaussianFlux', 'base_Na

^^^ these differences make sense.


I updated the measurment plugin overrides in response to these differences (we weren't picking up the results of the LATISS config sprint)
```
astrometry.sourceSelector.values.science.doIsolated True False
astrometry.sourceSelector.values.science.doRequirePrimary True False
doDeblend True False
measurement.slots.shape ext_shapeHSM_HsmSourceMoments base_SdssShape
measurement.slots.psfShape ext_shapeHSM_HsmPsfMoments base_SdssShape_psf
measurement.slots.apFlux base_CircularApertureFlux_35_0 base_CircularApertureFlux_12_0
measurement.slots.calibFlux base_CircularApertureFlux_35_0 base_CircularApertureFlux_12_0
measurement.plugins.names ['base_CircularApertureFlux', 'base_PsfFlux', 'ext_shapeHSM_HsmSourceMomentsRound', 'base_NaiveCentroid', 'base_CompensatedGaussianFlux', 'base_Blendedness', 'ext_photometryKron_KronFlux', 'base_LocalBackground', 'ext_shapeHSM_HsmPsfMomentsDebiased', 'base_SdssCentroid', 'ext_shapeHSM_HsmSourceMoments', 'base_SdssShape', 'base_Variance', 'base_Jacobian', 'base_PixelFlags', 'base_GaussianFlux', 'ext_shapeHSM_HsmShapeRegauss', 'base_SkyCoord', 'ext_shapeHSM_HsmPsfMoments', 'base_FPPosition'] ['base_CircularApertureFlux', 'base_PsfFlux', 'base_SdssShape', 'base_Variance', 'base_NaiveCentroid', 'base_PixelFlags', 'base_SdssCentroid', 'base_CompensatedGaussianFlux', 'base_Blendedness', 'base_GaussianFlux', 'base_LocalBackground', 'base_SkyCoord']
measurement.plugins.values.base_CircularApertureFlux.maxSincRadius 12.0 10.0
measurement.plugins.values.base_Jacobian.pixelScale 0.1 0.5
measurement.plugins.values.ext_shapeHSM_HsmShapeRegauss.deblendNChild deblend_nChild None
doApCorr True False
```